In [17]:
import xlrd
import sqlite3
import time

In [18]:
# Open the workbook and define the worksheet
BOOK = xlrd.open_workbook(filename="/Users/muhamuttaqien/Desktop/Al-Qur'an/LQ Tagging/ibnKathirTafsirs DB/IndeksQuranTercanggih.xlsx")

DEPTH_TOPIC_LEVEL = 6

LIST_ENTRY = ["IMAN", "ILMU", "BANGSA-BANGSA TERDAHULU", "SEJARAH", "AL QUR'AN", 
           "AKHLAQ & ADAB", "IBADAH", "MAKANAN & MINUMAN", "PAKAIAN & PERHIASAN", 
           "HUKUM PRIVAT", "MUAMALAT", "PERADILAN & HAKIM", "HUKUM PIDANA, JINAYAH", "JIHAD"]

In [19]:
conn = sqlite3.connect("/Users/muhamuttaqien/Documents/workspace/learnquran-tafsir-web/lqtafsir-server/app/resource/IbnKathirTafsirs-tagged-parented.db3")
c = conn.cursor()

In [20]:
sheet = BOOK.sheet_by_name("IMAN")

### Count rows and columns

In [21]:
table = "tafsir_id_content"
rowsQuery = "SELECT Count() FROM %s" % table
c.execute(rowsQuery)
numberOfRows = c.fetchone()[0]
numberOfRows

6236

In [22]:
columnsQuery = "PRAGMA table_info(%s)" % table
c.execute(columnsQuery)
numberOfColumns = len(c.fetchall())
numberOfColumns

5

In [23]:
nrows = sheet.nrows
nrows

475

In [24]:
ncols = sheet.ncols
ncols

253

### Clear all keywords (optional, hit only once)

In [25]:
for row in range(numberOfRows+1):
    c.execute('UPDATE tafsir_id_content SET keywords = "" WHERE docid = ' + str(row))
    
conn.commit()

### Detect topic and verses

In [26]:
topic = sheet.cell_value(rowx=2, colx=1)
topic

'Keutamaan dan pahala iman'

In [27]:
topic = sheet.cell_value(rowx=3, colx=2)
topic

'Kesentosaan orang mukmin di dunia dan di akhirat'

In [28]:
verse = sheet.cell_value(rowx=8, colx=7)
verse

'Qs.84:8'

In [29]:
# get verses
for pcol in range(7, ncols):
    colx = pcol
    verse = sheet.cell_value(rowx=2, colx=colx).strip()
    if(verse):
        print(verse)

### Get and Update Parents

In [30]:
def get_and_update_parents(prow, pcol, depth_level, surah, ayah):
    if (depth_level > 1):
        topic_parents = []
        for up_point in range(1, depth_level):
            topic = sheet.cell_value(rowx=(prow-up_point), colx=(pcol-up_point))
            if not topic:
                
                empty_up_point = up_point
                
                while(topic == ''): # check parents if empty looped up
                    topic = sheet.cell_value(rowx=(prow-empty_up_point), colx=(pcol-up_point))
                    empty_up_point += 1
                    
                topic = topic + ' ' + str(depth_level-up_point)
                topic_parents.append(topic)
                
            else:
                topic = topic + ' ' + str(depth_level-up_point)
                topic_parents.append(topic)
        
        topic_parents_string = ', '.join(topic_parents)
                
        print("topic_parents: ", topic_parents_string)
        return topic_parents_string

### Tag topics to verses

In [31]:
def tag_topics(entry, sheet, depth_level):
    
    nrows = sheet.nrows
    ncols = sheet.ncols
    
    for prow in range(nrows):
        pcol = depth_level # index where topics positioned
        # print("prow: ", prow)
        # print("pcol: ", pcol)
        topic = sheet.cell_value(rowx=prow, colx=pcol) # (level 1: Keutamaan dan pahala iman, Cabang-cabang iman, Hal-hal yang merusak iman)
        
        if(topic):
            print("*** topic: ", topic.strip())
            srow = prow+1
            
            # add hierarchy level to main topic
            topic = topic + " " + str(depth_level)
            
            for pcol_v in range(7, ncols):
                colx = pcol_v
                verse = sheet.cell_value(rowx=prow, colx=colx).strip() # (Qs.2:38, Qs.2:62, Qs.2:112)
                if(verse): 
                    # print(verse)
                    # insert verse to db
                    splitted_verse = verse[3:].split(":")
                    surah = splitted_verse[0]
                    ayah = splitted_verse[1]

                    print("Tag into Qs. "+ surah + ":"+ ayah)                      
                    
                    # set multiple tags
                    for row in c.execute('SELECT keywords FROM tafsir_id_content WHERE c0sura = '+ surah +' AND c1aya = ' + ayah):
                        if(row[0] == None or row[0] == ''):
                            topic_parents_string = get_and_update_parents(prow, pcol, depth_level, surah, ayah)
                            # print("=== Qs. "+ surah + ":"+ ayah)
                            # print("topic before: ", topic)
                            
                            if(topic_parents_string):
                                topic_verse = (topic) + (', ' + topic_parents_string)
                            else:
                                topic_verse = (topic)

                            # print("topic after: ", topic_verse)
                            c.execute('UPDATE tafsir_id_content SET keywords = "'+ topic_verse +'" WHERE c0sura = '+ surah +' AND c1aya = ' + ayah)
                        else:
                            topic_parents_string = get_and_update_parents(prow, pcol, depth_level, surah, ayah)
                            # print("=== Qs. "+ surah + ":"+ ayah)
                            # print("topic before: ", topic)
                            if(topic_parents_string):
                                topic_verse = (topic) + (', ' + topic_parents_string)
                            else:
                                topic_verse = (topic)
                                
                            joint_topic = row[0] + ", " + (topic_verse)

                            # print("joint_topic: ", joint_topic)
                            c.execute('UPDATE tafsir_id_content SET keywords = "'+ joint_topic +'" WHERE c0sura = '+ surah +' AND c1aya = ' + ayah)

    conn.commit()
    print("\n\n---------------------------------- tag %s level %s committed! ----------------------------------\n\n" % (entry, depth_level))

### Commit tagging!

#### all entries

In [32]:
start_time = time.time()

for entry in LIST_ENTRY:
    sheet = BOOK.sheet_by_name(entry)
    
    print("****************************************** %s ******************************************" % (entry))
    
    for level in range(1, (DEPTH_TOPIC_LEVEL+1)):
        tag_topics(entry, sheet, level)
    
print("--- Tagging process takes %.4f seconds ---" % (time.time() - start_time))

****************************************** IMAN ******************************************
*** topic:  Keutamaan dan pahala iman
*** topic:  Cabang-cabang iman
*** topic:  Hal-hal yang merusak iman
*** topic:  Iman bertambah dan berkurang


---------------------------------- tag IMAN level 1 committed! ----------------------------------


*** topic:  Kesentosaan orang mukmin di dunia dan di akhirat
Tag into Qs. 2:38
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:62
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:112
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:262
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:274
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:277
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 3:103
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 3:107
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 3:170
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 4:173
topic_parents:

Tag into Qs. 28:25
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 28:31
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 28:35
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 29:15
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 29:24
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 29:32
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 29:33
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 34:20
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 34:21
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 37:76
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 37:98
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 37:115
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 37:134
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 37:146
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 38:83
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 40:45
topic_parents:  Keutamaan dan pah

topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 8:12
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 9:26
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 9:40
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 14:27
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 16:102
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 47:7
topic_parents:  Keutamaan dan pahala iman 1
*** topic:  Pahala iman
Tag into Qs. 2:5
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:25
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:82
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:103
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:126
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:202
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:212
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:218
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 2:221
topic_parents:  Keutamaan dan pahala i

Tag into Qs. 39:74
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 40:40
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 40:51
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 41:8
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 41:18
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 41:30
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 41:31
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 41:32
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 42:22
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 43:35
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 43:70
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 43:72
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 43:73
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 44:51
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 44:52
topic_parents:  Keutamaan dan pahala iman 1
Tag into Qs. 44:53
topic_parents:  Keutamaan dan pahala 

Tag into Qs. 9:124
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 18:13
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 19:76
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 47:17
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 48:4
topic_parents:  Iman bertambah dan berkurang 1
*** topic:  Iman adalah ucapan dan perbuatan
Tag into Qs. 2:3
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 2:25
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 2:62
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 2:82
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 2:143
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 2:177
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 2:217
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 2:277
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 2:278
topic_parents:  Iman bertambah dan berkurang 1
Tag into Qs. 3:57
topic_parents:  Ima

...

Tag into Qs. 5:32
topic_parents:  Hukum membunuh 2, Kejahatan membunuh 1
*** topic:  Membunuh diharamkan
*** topic:  Pembunuhan dengan sengaja
Tag into Qs. 2:178
topic_parents:  Jenis-jenis pembunuhan 2, Kejahatan membunuh 1
Tag into Qs. 4:93
topic_parents:  Jenis-jenis pembunuhan 2, Kejahatan membunuh 1
*** topic:  Pembunuhan tidak sengaja
Tag into Qs. 4:92
topic_parents:  Jenis-jenis pembunuhan 2, Kejahatan membunuh 1
*** topic:  Kisas (hukuman balasan)
*** topic:  Diat (denda) pembunuhan
*** topic:  Kafarat membunuh
Tag into Qs. 4:92
topic_parents:  Sanksi membunuh 2, Kejahatan membunuh 1
*** topic:  Penyesalan si pembunuh dan taubatnya
Tag into Qs. 4:92
topic_parents:  Sanksi membunuh 2, Kejahatan membunuh 1
*** topic:  Kisas bagi yang melukai orang lain
Tag into Qs. 5:45
topic_parents:  Sanksi melukai orang lain 2, Kejahatan selain membunuh 1
*** topic:  Gugurnya hukuman melukai orang lain
Tag into Qs. 5:45
topic_parents:  Sanksi melukai orang lain 2, Kejahatan selain membunuh 1
*

*** topic:  Cara-cara mendera pelaku zina
Tag into Qs. 24:2
topic_parents:  Mendera pelaku zina 3, Sanksi berzina 2, Kejahatan berzina 1
*** topic:  Hukum memotong tangan pencuri
Tag into Qs. 5:38
topic_parents:  Pemotongan tangan pencuri 3, Sanksi mencuri 2, Kejahatan mencuri 1


---------------------------------- tag HUKUM PIDANA, JINAYAH level 4 committed! ----------------------------------


*** topic:  |||<Al Qur'an


---------------------------------- tag HUKUM PIDANA, JINAYAH level 5 committed! ----------------------------------




---------------------------------- tag HUKUM PIDANA, JINAYAH level 6 committed! ----------------------------------


****************************************** JIHAD ******************************************
*** topic:  Keutamaan jihad dan mujahidin (pejuang)
*** topic:  Hukum berjihad
*** topic:  Tidak ikut berperang
*** topic:  Sifat-sifat mujahidin
*** topic:  Persiapan berperang
*** topic:  Fase-fase pertempuran
*** topic:  Harta rampasan perang

topic_parents:  Keutamaan mujahidin (pejuang) 2, Keutamaan jihad dan mujahidin (pejuang) 1
Tag into Qs. 9:20
topic_parents:  Keutamaan mujahidin (pejuang) 2, Keutamaan jihad dan mujahidin (pejuang) 1
Tag into Qs. 9:41
topic_parents:  Keutamaan mujahidin (pejuang) 2, Keutamaan jihad dan mujahidin (pejuang) 1
Tag into Qs. 9:60
topic_parents:  Keutamaan mujahidin (pejuang) 2, Keutamaan jihad dan mujahidin (pejuang) 1
Tag into Qs. 22:58
topic_parents:  Keutamaan mujahidin (pejuang) 2, Keutamaan jihad dan mujahidin (pejuang) 1
Tag into Qs. 49:15
topic_parents:  Keutamaan mujahidin (pejuang) 2, Keutamaan jihad dan mujahidin (pejuang) 1
Tag into Qs. 57:10
topic_parents:  Keutamaan mujahidin (pejuang) 2, Keutamaan jihad dan mujahidin (pejuang) 1
*** topic:  Derajat pejuang
Tag into Qs. 4:95
topic_parents:  Keutamaan mujahidin (pejuang) 2, Keutamaan jihad dan mujahidin (pejuang) 1
Tag into Qs. 4:96
topic_parents:  Keutamaan mujahidin (pejuang) 2, Keutamaan jihad dan mujahidin (pejuang) 1
Tag in

topic_parents:  Sifat-sifat komandan / pemimpin 2, Sifat-sifat mujahidin 1
Tag into Qs. 4:76
topic_parents:  Sifat-sifat komandan / pemimpin 2, Sifat-sifat mujahidin 1
Tag into Qs. 4:84
topic_parents:  Sifat-sifat komandan / pemimpin 2, Sifat-sifat mujahidin 1
Tag into Qs. 8:60
topic_parents:  Sifat-sifat komandan / pemimpin 2, Sifat-sifat mujahidin 1
Tag into Qs. 8:65
topic_parents:  Sifat-sifat komandan / pemimpin 2, Sifat-sifat mujahidin 1
*** topic:  Musyawarat komandan dengan orang yang berpengalaman
Tag into Qs. 27:32
topic_parents:  Sifat-sifat komandan / pemimpin 2, Sifat-sifat mujahidin 1
Tag into Qs. 27:33
topic_parents:  Sifat-sifat komandan / pemimpin 2, Sifat-sifat mujahidin 1
*** topic:  Kendaraan tempur
*** topic:  Persiapan rohani dan jasmani mujahidin
Tag into Qs. 2:249
topic_parents:  Melatih mujahidin 2, Persiapan berperang 1
Tag into Qs. 4:84
topic_parents:  Melatih mujahidin 2, Persiapan berperang 1
Tag into Qs. 8:60
topic_parents:  Melatih mujahidin 2, Persiapan b

*** topic:  Gencatan senjata antara Nabi dan orang-orang musyrik
Tag into Qs. 8:61
topic_parents:  Gencatan senjata 2, Perjanjian 1
Tag into Qs. 9:4
topic_parents:  Gencatan senjata 2, Perjanjian 1


---------------------------------- tag JIHAD level 3 committed! ----------------------------------


*** topic:  Perang dengan menggunakan kuda
Tag into Qs. 100:1
topic_parents:  Kendaraan tempur 3, Peralatan tempur 2, Persiapan berperang 1
Tag into Qs. 100:2
topic_parents:  Kendaraan tempur 3, Peralatan tempur 2, Persiapan berperang 1
Tag into Qs. 100:3
topic_parents:  Kendaraan tempur 3, Peralatan tempur 2, Persiapan berperang 1
Tag into Qs. 100:4
topic_parents:  Kendaraan tempur 3, Peralatan tempur 2, Persiapan berperang 1
Tag into Qs. 100:5
topic_parents:  Kendaraan tempur 3, Peralatan tempur 2, Persiapan berperang 1
*** topic:  Besarnya pahala orang yang mati syahid
Tag into Qs. 3:169
topic_parents:  Keutamaan orang yang mati syahid 3, Mati syahid 2, Fase-fase pertempuran 1
Tag into Q

#### One by one

In [ ]:
LIST_ENTRY = ["IMAN", "ILMU", "BANGSA-BANGSA TERDAHULU", "SEJARAH", "AL QUR'AN", 
           "AKHLAQ & ADAB", "IBADAH", "MAKANAN & MINUMAN", "PAKAIAN & PERHIASAN", 
           "HUKUM PRIVAT", "MUAMALAT", "PERADILAN & HAKIM", "HUKUM PIDANA, JINAYAH", "JIHAD"]

start_time = time.time()
entry = "JIHAD"
sheet = BOOK.sheet_by_name(entry)

for level in range(1, (DEPTH_TOPIC_LEVEL+1)):
    tag_topics(entry, sheet, level)

print("--- Tagging process takes %.4f seconds ---" % (time.time() - start_time))